# Random Forrest Regression 

## Import Jobs and get Dataframe

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow.parquet as pq

#For Big Query
from google.cloud import bigquery     
from google.oauth2 import service_account

#For ML Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from scipy.stats import randint

#For Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz


In [24]:
# Connect to BQ

credentials = service_account.Credentials.from_service_account_file(
'C:/Users/miria/Desktop/music-recommendation-system-24-3d0d21fb1f8b.json')
# music-recommendation-system-24-3d0d21fb1f8b.json is the service account JSON file. Save the file locally on your device and add the path here. 
# make sure that the slashes in the path are '/' and not '\'

project_id = 'music-recommendation-system-24'
client = bigquery.Client(credentials= credentials,project=project_id)

In [25]:
# Query BQ

query_job = client.query("""
   SELECT 
      *
   FROM `music-recommendation-system-24.ml_tables_eu.song_list_obama_wo_duplicates_view`""")


results = query_job.result() # Wait for the job to complete.

rows = [dict(row) for row in results]

# Convert the list of dictionaries to a DataFrame
df_bq = pd.DataFrame(rows)

In [26]:
df_bq.dtypes


acousticness                float64
danceability                float64
duration_min                float64
energy                      float64
genres                       object
instrumentalness            float64
key_name                     object
liveness                    float64
loudness                    float64
mode                         object
speechiness                 float64
track_album_name             object
track_album_release_year     object
track_artist                 object
track_id                     object
track_name                   object
track_popularity              int64
tempo                         int64
valence                     float64
in_obama_playlist             int64
dtype: object

##Build Model

In [27]:
#split X and y
X = df_bq[['acousticness', 
           'danceability', 
           'duration_min', 
           'energy', 
           'instrumentalness',
           'key_name',
           'liveness',
           'loudness',
           'mode',
           'speechiness',
           'track_popularity',
           'tempo',
           'valence']]
#drop: 
# genres (too much difference between origninal tables), 
# track_album_name, 
# track_album_release_year (to much difference between orignal tables)
# track_id & track_name (no value for model)
# in_obama_playlist (is y)
y = df_bq[['in_obama_playlist']]

In [28]:
#split test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train.shape, y_train.shape

((19149, 13), (19149, 1))

In [29]:
# split numeric and non numeric columns for X_train and X_test
X_train_numeric = X_train.select_dtypes(include=['int64', 'float64'])
X_test_numeric = X_test.select_dtypes(include=['int64', 'float64'])

X_train_non_numeric = X_train.select_dtypes(exclude=['int64', 'float64'])
X_test_non_numeric = X_test.select_dtypes(exclude=['int64', 'float64'])

X_train_numeric.shape, X_train_non_numeric.shape, y_train.shape

((19149, 11), (19149, 2), (19149, 1))

In [30]:
#Classify non numeric data

X_train_non_numeric_ohe = enc.fit_transform(X_train_non_numeric)
X_test_non_numeric_ohe = enc.transform(X_test_non_numeric)

X_train_non_numeric_ohe = pd.DataFrame(X_train_non_numeric_ohe, columns=enc.get_feature_names_out())

X_test_non_numeric_ohe = pd.DataFrame(X_test_non_numeric_ohe, columns=enc.get_feature_names_out())

X_train_numeric.shape, X_train_non_numeric_ohe.shape, y_train.shape


((19149, 11), (19149, 14), (19149, 1))

In [37]:
# concat normalised X_train's and X_test's

X_train_normalised = pd.concat([X_train_numeric.reset_index(), X_train_non_numeric_ohe.reset_index()], axis=1)

X_test_normalised = pd.concat([X_test_numeric.reset_index(), X_test_non_numeric_ohe.reset_index()], axis=1)

X_test_normalised.shape, X_train_normalised.shape, y_test.shape, y_train.shape


((9433, 27), (19149, 27), (9433, 1), (19149, 1))

In [32]:
#start Random Forrest
rf.fit(X_train_normalised, y_train)

c:\Users\miria\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [42]:
#predict Model
y_pred = rf.predict_proba(X_test_normalised.iloc[[2]])

y_pred

array([[0.98, 0.02]])

In [46]:
y_test.value_counts

<bound method DataFrame.value_counts of        in_obama_playlist
24986                  0
3410                   0
9534                   0
13818                  0
20902                  1
...                  ...
16606                  0
3878                   0
24083                  0
8233                   0
22270                  0

[9433 rows x 1 columns]>

In [51]:
y_test['in_obama_playlist'].value_counts()

in_obama_playlist
0    9354
1      79
Name: count, dtype: int64

In [49]:
#Test Model
accuracy = accuracy_score(X_test_normalised, y_test['in_obama_playlist'])
print("Accuracy:", accuracy)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets